In [30]:
import pandas as pd
import numpy as np
import PyPDF2
import os
import re

In [29]:
import tika
from tika import parser

In [60]:
from tabula import read_pdf

# Get all the PDF filenames

In [2]:
pdf_files = []

In [3]:
for filename in os.listdir('Cvs/'):
    if filename.endswith('.pdf'):
         pdf_files.append('Cvs/'+filename)

# Number of pdf files collected

In [4]:
len(pdf_files)

456

In [9]:
from cStringIO import StringIO
import subprocess

input_path = 'c:/test_in.pdf'

def decompress_pdf(temp_buffer):
    temp_buffer.seek(0)  # Make sure we're at the start of the file.

    process = subprocess.Popen(['pdftk.exe',
                                '-',  # Read from stdin.
                                'output',
                                '-',  # Write to stdout.
                                'uncompress'],
                                stdin=temp_buffer,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    return StringIO(stdout)

ModuleNotFoundError: No module named 'cStringIO'

In [5]:
from PyPDF2 import utils
def extract_text_from_pdf():
    res = {}
    for pdf in pdf_files:
        pdfFileObj = open(pdf, 'rb')
        try:
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        except utils.PdfReadError:
            print(pdf, " not processed")
            # pdfReader = PyPDF2.PdfFileReader(decompress_pdf(input_file))
        if not pdfReader.isEncrypted:
            num_pages = pdfReader.numPages
            pages = []
            for page_nb in range(num_pages):
                pages.append(pdfReader.getPage(page_nb).extractText())
            res[pdf] = ' '.join(pages)
    return res

In [17]:
pdf_contents = extract_text_from_pdf()

Cvs/Hu_Xuan_20190320090349.pdf  not processed


Cvs/Wong_Terence_20190320085947.pdf  not processed


In [75]:
def clean_pdf():
    res = {}
    for name, content in pdf_contents.items():
        x = content.split('\n')
        x = [i for i in x if len(i)>1]
        x = [i.rstrip() for i in x if bool(re.search('[a-zA-Z]+', i))]
        n = len(x)
        for i, text in enumerate(x):
            if i > 0:
                if len(text) > 1:
                    if text[0].isupper() or text[1].isupper():
                        x[i] = '. ' + text
            if i < n - 1:
                next_text = x[i+1]
                first_word = next_text.split(' ')[0]
                if first_word.isdigit() or len(first_word)>1:
                    x[i+1] = ' ' + next_text
        pdf = ''.join(x)
        sentences = pdf.split('.')
        res[name] = sentences
    return res   

In [76]:
cleaned_pdf = clean_pdf()

In [77]:
cleaned_pdf[pdf_files[1]]

['Sachin Abeysundera 14 The Fairway, Oadby, Leicester LE2 2HJ   Mobile: 07528733306       Email:sachin97@live',
 'co',
 'uk',
 '  Personal Profile',
 ' I have studied Physiology at the University of Manchester, where I have recently graduated with a 2:1, and I am looking to secure a job, preferably in a position which can exploit my analytical and leadership skills, as well as improving and developing th ese skills',
 ' University has greatly enhanced my communication and written skills, highlighted by my dissertation mark of 68%',
 ' Furthermore, university has encouraged me to be much more independent, and has improved my time management skills, due to the number of deadlines I have had to meet during the last three years, which will stand me in good stead in the working environment',
 '',
 '  Education',
 '  University of Manchester',
 '  Leicester Grammar School levels: Biology (A*), Chemistry (A), Maths (A)',
 '  As Level',
 '  Religious Studies (A)',
 '  GCSEs: 10 GCSEs including

In [74]:
pdf_contents[pdf_files[1]]

'Sachin Abeysundera\n \n14 The Fairway, Oadby, Leicester LE2 2HJ   Mobile: 07528733306       Email:sachin97@live.co.uk\n \n \nPersonal Profile\n \n \nI have studied Physiology at the University of Manchester, where I have recently graduated with a \n2:1, and I am looking to secure a job, preferably in a position which can exploit my analytical and \nleadership skills, as well as improving and developing th\nese skills. University has greatly enhanced \nmy communication and written skills, highlighted by my dissertation mark of 68%. Furthermore, \nuniversity has encouraged me to be much more independent, and has improved my time \nmanagement skills, due to the numbe\nr of deadlines I have had to meet during the last three years, \nwhich will stand me in good stead in the working environment. \n \n \nEducation \n \nUniversity of Manchester\n \n(2015\n-\n2018) \n-\n \n2:1\n \n \nLeicester Grammar School\n \n(2007\n-\n2015) \nA\n-\nlevels\n: Biology (A*), Chemistry (A)\n, Maths (A) \nAs Le